# Notebook Objective and Setup

BGG01 involves the acquisition of game data from BoardGameGeek. Largely this is accomplished by XML API call, with some dynamic content scraped. Files are dumped to a "dirty" directory.

## Package Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from bs4 import BeautifulSoup
import requests
import regex as re
import time
import json
import os
import gc

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os


## Functions

In [2]:
def create_designers(game_page, game_id):
    '''Create DataFrame for Designers for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all designers on page
    all_designers = game_page.find_all('link', type='boardgamedesigner')
    
    # make dataframe
    designers = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    design = {'BGGId':int(game_id)}
    
    # add this item's designers to dictionary
    for item in all_designers:
        design[item['value']] = int(1)
    
    # append to dataframe
    designers = designers.append(design, ignore_index=True)
    
    # return dataframe
    return designers

In [3]:
def create_categories(game_page, game_id):
    '''Create DataFrame for Categories for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all categories on page
    all_categories = game_page.find_all('link', type='boardgamecategory')
    
    # make dataframe
    categories = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    category = {'BGGId':int(game_id)}
    
    # add this item's categories to dictionary
    for item in all_categories:
        category[item['value']] = int(1)
    
    # append to dataframe
    categories = categories.append(category, ignore_index=True)
    
    # return dataframe
    return categories

In [4]:
def create_mechanics(game_page, game_id):
    '''Create DataFrame for Mechanics for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all mechanics on page
    all_mechanics = game_page.find_all('link', type='boardgamemechanic')
    # make dataframe
    mechanics = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    mechanic = {'BGGId':int(game_id)}

    # add this item's mechanics to dictionary
    for item in all_mechanics:
        mechanic[item['value']] = int(1)
    
    # Try Tableau
    try:
        game_page.find('link', type='boardgamefamily', value=("Mechanism: Tableau Building"))['value']
        mechanic['TableauBuilding'] = int(1)
    except: pass
    
    # Try is Legacy
    try:
        game_page.find('link', type='boardgamefamily', value=("Mechanism: Legacy"))['value']
        mechanic['Legacy'] = int(1)
    except: pass
    
    # append to dataframe
    mechanics = mechanics.append(mechanic, ignore_index=True)
    # return dataframe
    return mechanics

In [5]:
def create_artists(game_page, game_id):
    '''Create DataFrame for artists for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all artists on page
    all_artists = game_page.find_all('link', type='boardgameartist')
    
    # make dataframe
    artists = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    artist = {'BGGId':int(game_id)}

    # add this item's artists to dictionary
    for item in all_artists:
        artist[item['value']] = int(1)
    
    # append to dataframe
    artists = artists.append(artist, ignore_index=True)
    
    # return dataframe
    return artists

In [6]:
def create_publishers(game_page, game_id):
    '''Create DataFrame for Mechanics for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all publishers on page
    all_publishers = game_page.find_all('link', type='boardgamepublisher')
    
    # make dataframe
    publishers = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    publisher = {'BGGId':int(game_id)}
    
    # add this item's artists to dictionary
    for item in all_publishers:
        publisher[item['value']] = int(1)
    
    # append to dataframe
    publishers = publishers.append(publisher, ignore_index=True)
    
    # return dataframe
    return publishers

In [7]:
def create_awards(awards_level, game_id):
    '''Create DataFrame for Awards for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all awards on page
    all_awards = awards_level.find_all('a', class_='ng-binding')
    
    # make dataframe
    awards = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    award = {'BGGId':int(game_id)}

    # add this item's awards to dictionary
    for item in all_awards:
        item = re.sub("[0-9]", "", item.text).strip(' ')
        award[item] = int(1)
    
    # append to dataframe
    awards = awards.append(award, ignore_index=True)
    
    # return dataframe
    return awards

In [8]:
def create_ratings_dist(stats_page, game_id):
    '''Create DataFrame for ratings ditribution for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe
    '''
    
    # find all ratings sections
    all_ratings = stats_page.find('ratings-stats-graph')
    # find all ratings entries
    next_ratings = all_ratings.find_all('text')
    
    # make dataframe
    ratings = pd.DataFrame(columns=['BGGId'])
    # make dictionary
    rating = {'BGGId':int(game_id)}
    
    # set all ratings in dictionary
    rating['1'] = next_ratings[10].text
    rating['2'] = next_ratings[11].text
    rating['3'] = next_ratings[12].text
    rating['4'] = next_ratings[13].text
    rating['5'] = next_ratings[14].text
    rating['6'] = next_ratings[15].text
    rating['7'] = next_ratings[16].text
    rating['8'] = next_ratings[17].text
    rating['9'] = next_ratings[18].text
    rating['10'] = next_ratings[19].text
    
    # append to dataframe
    ratings = ratings.append(rating, ignore_index=True)
    
    # return dataframe
    return ratings

# PULL - Game Data

In [ ]:
# set up our columns list
columns = ['BGGId',
                'Name',
               'Description',
                'YearPublished',
                'GameWeight',
                'AvgRating',  
                'BayesAvgRating',
                'StdDev',
                'MinPlayers',
                'MaxPlayers',
                'ComAgeRec',
                'LanguageEase',
                'BestPlayers',
                'GoodPlayers',
                'NumOwned',
                'NumWant',
                'NumWish',
                'NumWeightVotes',
                'MfgPlaytime',
                'ComMinPlaytime',
                'ComMaxPlaytime',
                'MfgAgeRec',
                'NumUserRatings',
                'NumComments',
                'NumAlternates',
                'NumExpansions',
                'NumImplementations',
           'IsReimplementation',
                'Family',
                'Theme',
               'Category',
               'Kickstarted',
               'ImagePath',
          ]

Last game id: 349161

In [ ]:
game_ids_current = pd.read_pickle('data_cleaned/game_ids_current')
game_ids = list(game_ids_current)

In [ ]:
start_position =0
end_position = 1000
file_suffix = 0

overall_start = time.time()
while end_position < (len(game_ids)+1):
    
    games = pd.DataFrame(columns=columns)
    designers = pd.DataFrame(columns=['BGGId'])
    categories = pd.DataFrame(columns=['BGGId'])
    mechanics = pd.DataFrame(columns=['BGGId'])
    artists = pd.DataFrame(columns=['BGGId'])
    publishers = pd.DataFrame(columns=['BGGId'])
    subcategories = pd.DataFrame(columns=['BGGId'])
    comments = pd.DataFrame(columns=['BGGId'])
    
    ##### File Setup Section #####
    
    # increment file suffix
    file_suffix += 1
    # get file suffix as string
    suffix_str = str(file_suffix)
    
    # print start and end positions
    print("Getting items "+str(start_position+1)+' through '+str(end_position))
    
    # get list of game ids to grab
    #grab_list = game_ids[0][start_position:end_position]
    grab_list = game_ids[start_position:end_position]
    
    # piece together target string of game ids for BGG 
    targets = ''
    for item in grab_list:
        targets += str(item)+','
    
    # log start time for information retrieval
    start = time.time()# log the start time for this entry   
    
    
    ##### API Call Section #####
    
    # Set up Selenium drivers
    options = webdriver.ChromeOptions() # set up chrome options
    options.add_argument("--headless") # set up chrome options
    time.sleep(1) # wait 1 second
    # establish path with targets
    path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&stats=1&type=boardgame'#&comments=1'&ratingcomments=1&page=1&pagesize=100
    driver = webdriver.Chrome(options=options)# initiate chrome driver with options
    print("New page retrieval. May be waiting for load.")
    driver.get(path)# get path
    # wait until the driver finds the element that we need
    element = WebDriverWait(driver, 180).until(EC.presence_of_all_elements_located((By.ID, 'folder0')))
        
    game_page = BeautifulSoup(driver.page_source) # parse page with beautifulsoup    
    
    # make entry for each game item on page
    game_entries = game_page.find_all('item')
    
    print("Items loaded. Processing.")
    ##### Process Each Game #####
    
    for entry in game_entries:
        ##### Get Game Name, BGGId, and check that game should be included in list #####

        ##### Check is expansion #####
        #gametype = entry['type'] # check game type
        #if gametype != 'boardgame':
        #    continue
        #else: pass
       

        # check that this game has sufficient user ratings to incluide
        try:
            user_ratings = int(entry.find('usersrated')['value'])# get the number of user ratings
        
            if user_ratings < 30: #check if user ratings are under 30
                continue
        except: continue
            
        # get game name and BGG ID
        game_name = entry.find('name', type='primary')['value']
        game_id = entry['id']
        #print("Name: "+game_name+", BGG ID: "+str(game_id))

        
        ##### Get Basic Stats #####

        #print("Getting basic stats")
        description = entry.find('description').text # description text of the game
        
        try:
            year_pub = int(entry.find('yearpublished')['value']) # year published
            if year_pub > 2021:
                continue
        except: pass
            
        try: minplayers = int(entry.find('minplayers')['value']) # minimum players
        except: minplayers = None
            
        try: maxplayers = int(entry.find('maxplayers')['value']) # maximum players
        except: maxplayers = None
            
        avg_rating = float(entry.find('average')['value']) # average rating
        bayes_avg = float(entry.find('bayesaverage')['value']) # bayes average rating
        std_dev = float(entry.find('stddev')['value']) # standard deviation of rating
        num_owned = int(entry.find('owned')['value']) # num of people own this game
        num_want = int(entry.find('wanting')['value']) # num of people want this game
        num_wish = int(entry.find('wishing')['value']) # num of people with game on wishlist
        num_weight_votes = int(entry.find('numweights')['value']) # num of votes for game weight
        game_weight = float(entry.find('averageweight')['value']) # voted game weight
        
        try: image_path = entry.find('image').text # path to image
        except: image_path = None
            
        try:  mfg_play_time = int(entry.find('playingtime')['value']) # mfg stated playtime
        except: mfg_play_time = None
        try: comm_min_play = int(entry.find('minplaytime')['value']) # community min playtime
        except: comm_min_play = None
            
        try: comm_max_play = int(entry.find('maxplaytime')['value']) # community max playtime
        except: comm_max_play = None
        
        try: mfg_age = int(entry.find('minage')['value']) # mfg min age
        except: mfg_age = None
            
        #num_comments = int(entry.find('comments')['totalitems']) # num of ratings comments
        num_alts = len(entry.find_all('name', type='alternate')) # number alternate versions
        num_expansions = len(entry.find_all('link', type='boardgameexpansion')) # number of expansions
        num_implementations = len(entry.find_all('link', type='boardgameimplementation')) # number of implementations    
        

        
        ##### Get reimplementation flag #####
        reimplementation = entry.find('link', type="boardgameimplementation", inbound="true") # check if game is a reimplementation
        if reimplementation: 
            reimplements = 1 # if it's a reimplementation, flag it 1
        else: 
            reimplements = 0
 


        ##### Basic stats requiring some compaction/refinement #####

        # community age min
        try:
            age_poll = entry.find('poll', title="User Suggested Player Age").find_all('result')

            total = 0
            items = 0
        
            for item in age_poll:   
                vote = int(item['numvotes']) * int(item['value'][:2])
                total += vote
                items += int(item['numvotes'])

            if items>0: comm_age = total/items # make sure not dividing by 0, get community recommended age
            else: comm_age=None # if no votes, record none
        except: comm_age=None
        
        # Language Ease
        try:
         
            lang_poll = entry.find('poll', title="Language Dependence").find_all('result')
            total, items = 0, 0

            for item in lang_poll:   
                vote = int(item['numvotes']) * int(item['level'])
                total += vote
                items += int(item['numvotes'])

            if items>0: lang_ease = total/items # make sure not dividing by 0, get community language ease
            else: lang_ease=None # if no votes, record none
        except: lang_ease=None # if no votes, record none
            
        try:    
            # Best and Good Players
            players = entry.find('poll', title="User Suggested Number of Players").find_all('results') # get user players poll
            player_num_votes = int(entry.find('poll', title="User Suggested Number of Players")['totalvotes'])# get total votes
        
            best_players, best_score, good_players = 0, 0, [] # set up for best players loop
        
            if player_num_votes > 30: # evaluate if more than 30 votes for num players
                for player in players:
                    best = int(player.find('result', value='Best')['numvotes'])
                    rec = int(player.find('result', value='Recommended')['numvotes'])
                    score = best*2 + rec*1
                    positives = best+rec
                    ratio = positives/player_num_votes
                    if score > best_score: best_players, best_score = player['numplayers'], score # put in # players for best score
                    if ratio > .5: good_players.append(player['numplayers']) # put in good players if over 50% ratio
            else: best_players=None
        except: best_players=None
        
        ##### Skip dynamic content which cannot be batched #####
            
        #this_game['NumFans']=int(num_fans),
        #this_game['NumPageViews']=int(num_views),
        #this_game['RulesPosts']=int(rules_threads),
        #this_game['TotalPosts']=int(total_threads),            
        #this_game['NumAwards'] = int(num_awards)              
            
        
        # make dataframe for this game
        this_game = pd.DataFrame()
        this_game['BGGId']=int(game_id),
        this_game['Name']=game_name,
        this_game['Description']=description,
        this_game['YearPublished']=int(year_pub),
        this_game['GameWeight']=float(game_weight),
        this_game['AvgRating']=float(avg_rating),
        this_game['BayesAvgRating']=float(bayes_avg),
        this_game['StdDev']=float(std_dev),
        this_game['MinPlayers']=minplayers,
        this_game['MaxPlayers']=maxplayers,
        try: this_game['ComAgeRec']=float(comm_age),
        except: this_game['ComAgeRec']=None,
        try: this_game['LanguageEase']=float(lang_ease),
        except: this_game['LanguageEase']=None,
        this_game['BestPlayers']=best_players,
        this_game['GoodPlayers']=good_players,
        this_game['NumOwned']=int(num_owned),
        this_game['NumWant']=int(num_want),
        this_game['NumWish']=int(num_wish),
        this_game['NumWeightVotes']=int(num_weight_votes),
        this_game['MfgPlaytime']=mfg_play_time,
        this_game['ComMinPlaytime']=comm_min_play,
        this_game['ComMaxPlaytime']=comm_max_play,
        this_game['MfgAgeRec']=mfg_age,
        this_game['NumUserRatings']=int(user_ratings),
        #this_game['NumComments']=int(num_comments),
        this_game['NumAlternates']=int(num_alts),
        this_game['NumExpansions']=int(num_expansions),
        this_game['NumImplementations']=int(num_implementations),
        this_game['IsReimplementation']=int(reimplements),
        this_game['ImagePath']=image_path
            
        
        # add unique information to end of df
        
        # Add game ranks
        ranks = entry.find_all('rank')
        try:
            for item in ranks:
                this_game['Rank:'+item['name']] = float(item['value'])
        except: pass
        
        # Try to add components
        try: 
            families = entry.find_all('link', type='boardgamefamily', value=re.compile("Component"))
            for item in families:                    
                this_game['Components:'+item['name']] = item['value']
        except: pass
            
        
        # Try to add game series/family
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Game:"))['value'].strip('Game:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Series:"))['value'].strip('Series:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            setting = entry.find('link', type='boardgamefamily', value=re.compile("Setting:"))['value'].strip('Setting:').strip(' ')
            this_game['Setting'] = setting
        except: pass
            
        
        # Try to add theme
        try:
            theme = entry.find('link', type='boardgamefamily', value=re.compile("Theme:"))['value'].strip('Theme:').strip(' ')
            this_game['Theme'] = theme
        except: pass
            
        try:
            mechanism = entry.find('link', type='boardgamefamily', value=re.compile("Mechanism:"))['value'].strip('Mechanism:').strip(' ')
            this_game['Mechanism'] = mechanism
        except: pass
        
        # Try to add game category
        try:
            category = entry.find('link', type='boardgamefamily', value=re.compile("Category:"))['value'].strip('Category:').strip(' ')
            this_game['Category'] = category
        except: pass
        
        
        # Try is Kickstarted
        try:
            entry.find('link', type='boardgamefamily', value=re.compile("Crowdfunding"))['value']
            this_game['Kickstarted'] = int(1)
        except: pass
     
    
    
        ##### Get subcategories #####
        
        all_subcategories = entry.find_all('link', type='boardgamecategory')
        
        categories_hold = pd.DataFrame(columns=['BGGId'])
        subcategory = {'BGGId':int(game_id)}
    
        for item in all_subcategories:
            subcategory[item['value']] = int(1)      
        
        categories_hold = categories_hold.append(subcategory, ignore_index=True)    
        
        
        # create specialty dataframes
        designer = create_designers(entry, game_id)
        category = create_categories(entry, game_id)
        mechanic = create_mechanics(entry, game_id)
        artist = create_artists(entry, game_id)
        publisher = create_publishers(entry, game_id)
            
        games = games.append(this_game, ignore_index = True)
        designers = designers.append(designer, ignore_index=True)
        categories = categories.append(category, ignore_index=True)
        mechanics = mechanics.append(mechanic, ignore_index=True)
        artists = artists.append(artist, ignore_index=True)
        publishers = publishers.append(publisher, ignore_index=True)
        subcategories = subcategories.append(categories_hold, ignore_index=True)
        
    games.to_pickle('data_dirty/pulled_games/games'+suffix_str+'.pkl')
    designers.to_pickle('data_dirty/pulled_games/designers'+suffix_str+'.pkl')
    categories.to_pickle('data_dirty/pulled_games/categories'+suffix_str+'.pkl')
    mechanics.to_pickle('data_dirty/pulled_games/mechanics'+suffix_str+'.pkl')
    artists.to_pickle('data_dirty/pulled_games/artists'+suffix_str+'.pkl')
    publishers.to_pickle('data_dirty/pulled_games/publishers'+suffix_str+'.pkl')
    subcategories.to_pickle('data_dirty/pulled_games/subcategories'+suffix_str+'.pkl')
    
    print("Finished items in this group")
    
    print(f'Time: {time.time() - start}\n\n')       
    
    start_position += 1000
    end_position += 1000
    
print(f'Time: {time.time() - overall_start}\n\n') 

## DataValidation

In [ ]:
subcategories1 = pd.read_pickle('data_dirty/pulled_games/subcategories52.pkl')
games1 = pd.read_pickle('data_dirty/pulled_games/games52.pkl')
designers1 = pd.read_pickle('data_dirty/pulled_games/designers52.pkl')
categories1 = pd.read_pickle('data_dirty/pulled_games/categories52.pkl')
mechanics1 = pd.read_pickle('data_dirty/pulled_games/mechanics52.pkl')
artists1 = pd.read_pickle('data_dirty/pulled_games/artists52.pkl')
publishers1 = pd.read_pickle('data_dirty/pulled_games/publishers52.pkl')

In [ ]:
subcategories1

In [ ]:
games1

In [ ]:
designers1

In [ ]:
categories1

In [ ]:
mechanics1

In [ ]:
artists1

In [ ]:
publishers1

## Combine Files

In [ ]:
#games = pd.DataFrame(columns=columns)
#designers = pd.DataFrame(columns=['BGGId'])
#categories = pd.DataFrame(columns=['BGGId'])
#mechanics = pd.DataFrame(columns=['BGGId'])
#artists = pd.DataFrame(columns=['BGGId'])
#publishers = pd.DataFrame(columns=['BGGId'])
#subcategories = pd.DataFrame(columns=['BGGId'])

In [ ]:
for number in range(1, 354):
    print(number)
    
    #this_games = pd.read_pickle('data_dirty/pulled_games/games'+str(number)+'.pkl')
    #this_designers = pd.read_pickle('data_dirty/pulled_games/designers'+str(number)+'.pkl')
    #this_categories = pd.read_pickle('data_dirty/pulled_games/categories'+str(number)+'.pkl')
    #this_mechanics = pd.read_pickle('data_dirty/pulled_games/mechanics'+str(number)+'.pkl')
    #this_artists = pd.read_pickle('data_dirty/pulled_games/artists'+str(number)+'.pkl')
    #this_publishers = pd.read_pickle('data_dirty/pulled_games/publishers'+str(number)+'.pkl')
    #this_subcategories = pd.read_pickle('data_dirty/pulled_games/subcategories'+str(number)+'.pkl')
    
    #games = games.append(this_games)
    #designers = designers.append(this_designers)
    #categories = categories.append(this_categories)
    #mechanics = mechanics.append(this_mechanics)
    #artists = artists.append(this_artists)
    #publishers = publishers.append(this_publishers)
    #subcategories = subcategories.append(this_subcategories)
    
    

In [ ]:
#games = games.reset_index(drop=True)
#designers = designers.reset_index(drop=True)
#categories = categories.reset_index(drop=True)
#mechanics = mechanics.reset_index(drop=True)
#artists = artists.reset_index(drop=True)
#publishers = publishers.reset_index(drop=True)
#subcategories = subcategories.reset_index(drop=True)

In [ ]:
#games.to_pickle('data_dirty/games.pkl')
#designers.to_pickle('data_dirty/designers.pkl')
#categories.to_pickle('data_dirty/categories.pkl')
#mechanics.to_pickle('data_dirty/mechanics.pkl')
#artists.to_pickle('data_dirty/artists.pkl')
#publishers.to_pickle('data_dirty/publishers.pkl')
#subcategories.to_pickle('data_dirty/subcategories.pkl')

In [ ]:
break

# PULL - User Ratings

## Create Scraper URLs

In [9]:
game_ids_current = pd.read_pickle('data_cleaned/game_ids_current.pkl')
game_ids = list(game_ids_current)

games = pd.read_pickle('data_cleaned/games.pkl')

In [10]:
ratings_totals = pd.DataFrame(games['BGGId'])
ratings_totals['RatingsPages'] = np.ceil(games['NumUserRatings']/100).astype('int')
ratings_totals = ratings_totals.sort_values('RatingsPages', ascending=False).reset_index(drop=True)

In [12]:
ratings_totals[:500].head(30)

,BGGId,RatingsPages
0,30549,1082
1,822,1080
2,13,1072
3,68448,893
4,36218,812
5,9209,756
6,178900,736
7,167791,731
8,173346,683
9,31260,659


In [ ]:
group1 = ratings_totals[:1000]
group2 = ratings_totals[1000:2000]
group3 = ratings_totals[2000:3000]
group4 = ratings_totals[3000:4000]
group5 = ratings_totals[4000:5000]
group6 = ratings_totals[5000:6000]
group7 = ratings_totals[6000:7000]
group8 = ratings_totals[7000:8000]
group9 = ratings_totals[8000:9000]
group10 = ratings_totals[9000:10000]
group11 = ratings_totals[10000:11000]
group12 = ratings_totals[11000:12000]
group13 = ratings_totals[12000:13000]
group14 = ratings_totals[13000:14000]
group15 = ratings_totals[14000:15000]
group16 = ratings_totals[15000:16000]
group17 = ratings_totals[16000:17000]
group18 = ratings_totals[17000:18000]
group19 = ratings_totals[18000:19000]
group20 = ratings_totals[19000:20000]
group21 = ratings_totals[20000:21000]
group22 = ratings_totals[21000:]

In [ ]:
groups = [group1, group2, group3, group4, group5, group6, group7, group8, group9, group10,
         group11, group12, group13, group14, group15, group16, group17, group18, group19, group20,
         group21, group22]

In [ ]:
def generate_urls(group):
    urls_list = []
    
    max_pages = group.RatingsPages.max()
    min_pages = group.RatingsPages.min()
    pages = np.arange(max_pages,0, -1)
    max_size = group.shape[0]-1
    #print(max_pages, max_size)
    
    targets = ''
    current_index = 0
    
    for page in pages:
        
        current_thresh = page
        
        while group.iloc[current_index]['RatingsPages'] == current_thresh:
            
            # get BGGId for the index
            current_item = group.iloc[current_index]['BGGId']            
            
            # target is the current item
            target = str(current_item)   
            # add the target to the targets list
            targets += target+','            
            
            current_index += 1
            if current_index == 999 or current_index == max_size:
                # get BGGId for the index
                current_item = group.iloc[current_index]['BGGId']            
            
                # target is the current item
                target = str(current_item)   
                # add the target to the targets list
                targets += target+','
                break
        
        #print(current_index)
        
        # establish path with targets and current page
        path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&ratingcomments=1&page='+str(page)+'&pagesize=100'
        urls_list.append(path)
    
    return urls_list

In [ ]:
group_urls = {}
group_num = 0

for group in groups:
    group_num += 1
    print(group_num)
    group_urls["group"+str(group_num)] = generate_urls(group)
    
with open('data_cleaned/scraper_urls_ratings.json', 'w') as convert_file:
    convert_file.write(json.dumps(group_urls))

## Scrape URLs

In [ ]:
import scrapy

for item in group_urls:
    
    print(item)
    
    !scrapy crawl bgg -a group=$item

# Appendix

## Data Validation

In [ ]:
games=pd.read_pickle('data_dirty/games.pkl')
designers=pd.read_pickle('data_dirty/designers.pkl')
categories=pd.read_pickle('data_dirty/categories.pkl')
mechanics=pd.read_pickle('data_dirty/mechanics.pkl')
artists=pd.read_pickle('data_dirty/artists.pkl')
publishers=pd.read_pickle('data_dirty/publishers.pkl')
subcategories=pd.read_pickle('data_dirty/subcategories.pkl')

In [ ]:
games

In [ ]:
designers.tail()

In [ ]:
categories.tail()

In [ ]:
mechanics.tail()

In [ ]:
artists.tail()

In [ ]:
publishers.tail()

In [ ]:
subcategories.tail()

## Get game ids

In [ ]:
games_ids_current = games['BGGId']
games_ids_current

In [ ]:
games_ids_current.to_pickle('data_cleaned/game_ids_current.pkl')